TBM定義
===

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time
 
def includes(phi,x):
  return set(phi).issubset(x)

class TBM:
  def __init__ (self, n, B, S, X):
    
    self.set_B(B)
    self.set_S(S)
    self.n_= n
    self.set_Phi(self.S_,self.B_)
    self.set_Ssub(self.S_,self.B_)
    self.set_Xsub(X)
  def fit(self, X, n_iter, stepsize):
    """Actual implementation of TBM fitting.
        Parameters
        ----------
        X : array-like, shape (n_samples,)
            Training vectors, where n_samples is the number of samples and 
            each element is a tuple of indices of 1s.
        n_iter: number of iteration
        method: "grad"  = gradient descent, "coor" coordinate descent
        Returns
        -------
        self : object
    """

    self.compute_Phat(X)
    #print(self.Phat)
    self.set_etahat(X)
    #print(self.etahat_)
    print('gradient_descent')
    self.gradient_descent(X,n_iter, stepsize)
    
    return self
  
  def set_Xsub(self, X):
    self.Xsub = list(dict.fromkeys(X))
    self.invXsub = {}
    for i in range(len(self.Xsub)):
      self.invXsub[self.Xsub[i]] = i
  
  def set_B(self, B):
    """
    Generate B_ and invB_
    Parameters
    -----
    B : Set of parameters
    
    invB_ : dict type
               indices are vectors of set_B
    """
    self.B_ = B
    self.invB_ ={}
    for i in range(len(B)):
      self.invB_[B[i]] = i
      

  def set_S(self, S):
    """
    Generate S_ and invS_
    Parameters
    -----
    S : Sample space
    
    inv_S : dict type
               indices are vectors of set_S
    """
    self.S_ = S
    self.invS_ ={}
    #print(self.S_)
    #print(self.invS_)
    for i in range(len(S)):
      self.invS_[S[i]] = i
      
  
  def set_Phi(self, S, B):
    """
    Phi_[x]= { phi in B | x includes phi }
    """
    self.Phi_ = {}
    for x in S:
      Phix = []
      for phi in B:
        if includes(phi,x):
          Phix.append(phi)
      self.Phi_[x] = Phix

  def set_Ssub(self, S, B):
    """
    Ssub [phi]= { x in S | x includes phi } 
    """
    self.Ssub_ = {}
    for phi in B:
      ssub = []
      for x in S:
        if includes(phi,x):
          ssub.append(x)
      self.Ssub_[phi] = ssub

  def compute_Phat(self, X):
    self.Phat = np.zeros(len(self.Xsub))
    for xi in X:
        self.Phat[self.invXsub[xi]] += 1 / len(X)
    
  def set_etahat(self, X):
    """
        
        Parameters
        ----------
        X : array-like, shape (n_samples,)
            Training vectors, where n_samples is the number of samples and 
            each element is a tuple of indices of 1s.
            
    """
    self.etahat_ = {}
    for phi in self.B_:
      denom = 0.0
      for i in range(len(X)):
        denom += 1 if includes(phi, X[i]) else 0
      self.etahat_[phi] = denom / len(X)
    #print(self.etahat_)
          
  def init_theta(self):
    
    self.theta_ = np.zeros(len(self.B_))
    self.compute_theta_perp() 
    #print(self.theta_)
  
  def compute_theta_perp(self):
    """
    Computing theta_perp
    
    Returns
    -----
    theta_
    """
    r = 0.0 
    for x in self.S_:
      s = 0.0 # sum theta_phi phi(x) for theta in B minus perp
      for phi in self.Phi_[x]:
        if phi: # is not empty
          s += self.get_theta(phi)  
      r += np.exp(s)
    self.theta_[self.invB_[()]] = -np.log(r)      

  def get_theta(self, phi):
    """
    Getting theta
    Parameters 
    -----
    phi : element of set_B
    
    Returns
    -----
    ret : theta
    """
    return self.theta_[self.invB_[phi]]
  
  def set_theta(self, phi, value):
    
    self.theta_[self.invB_[phi]] = value
    
  
  def compute_P(self):
    """
    Computing P
    Returns
    -----
    P_ : len(P_) = len(S_)
    """
    self.P_ = np.zeros(len(self.S_))
    for x in self.S_:
      self.P_[self.invS_[x]] = np.exp( self.compute_logP(x) )
    #print(self.P_)
   

  def compute_logP(self, xi):
    """
    Computing lopP
    Parameters
    -----
    xi : i th row vector of X
    
    Returns
    -----
    ret : logP
    
    """
    ret = 0.0;
    for phi in self.B_ :
      ret +=  self.get_theta(phi) if includes(phi, xi) else 0    
    return ret 
      
  def compute_eta(self):
    """
    Computing eta
    Returns
    -----
    eta_ : dict type
             len(eta_) = len(B_)
    """
    self.eta_ = {}
    for phi in self.B_ :
      self.eta_[phi] = 0.0
      for x in self.Ssub_[phi] :
        self.eta_[phi] += self.P_[self.invS_[x]]
        
  def compute_ll(self, X):
    """
      Compute log likelihood
      Parameters
      ----------
      X : array-like, shape = [n_samples, n_features]

      Returns
      ----------
      ret :  log-likelihood

    """
    ret = 0.0
    #for i in range(len(X)):
    #  ret += self.compute_logP(X[i])
    for xi in X:
      ret += self.compute_logP(xi)
    return ret
  
  def compute_KL(self):
    """
    Computing KL_divergence
    
    Parameter
    -----
    X : array-like, shape (n_samples,)
            Training vectors, where n_samples is the number of samples and 
            each element is a tuple of indices of 1s. 
    -----
    """
    ret = 0.0
    for xi in self.Xsub:
      ret += self.Phat[self.invXsub[xi]] * (np.log(self.Phat[self.invXsub[xi]]) - self.compute_logP(xi))
    
    return ret
  
  def compute_squared_gradient(self):
      ret = 0.0
      for phi in self.B_:
#      if phi: # is not empty 
        ret += (self.etahat_[phi] - self.eta_[phi] ) ** 2
      
      return ret


  def gradient_descent(self, X, max_epoch, step):  
    """
    Actual implementation gradient_descent
    Parameters 
    -----
    X : array-like, shape (n_samples,)
            Training vectors, where n_samples is the number of samples and 
            each element is a tuple of indices of 1s. 
    max_epoch
    step 
    """
    self.init_theta()
    start = time.time()
    for epoch in range(max_epoch):
      self.compute_P()
      self.compute_eta()
      #print(self.compute_ll(X))
      print(epoch ,":",  "KL divergence:",f'{self.compute_KL():.8f}' ," time : %4.2f"% (time.time()-start),"Squared Gradient:",self.compute_squared_gradient())
      for phi in self.B_:
        if phi: # is not empty
          new_theta_phi = self.get_theta(phi) + step * (self.etahat_[phi] - self.eta_[phi] )
          self.set_theta(phi, new_theta_phi)
      self.compute_theta_perp()


  def coordinate_descent(self, X, max_epoch, step_init=2):
    """
    Actual implementation coodinate_descent
    Parameters 
    -----
    X : array-like, shape (n_samples,)
            Training vectors, where n_samples is the number of samples and 
            each element is a tuple of indices of 1s.
    max_epoch
    -----
    
    """
    u = {}
    for x in self.S_:
      u[x] = 1.0
    Z = len(self.S_)
    self.theta_ = np.zeros(len(self.B_)) 
    start = time.time()
    
    for epoch in range(max_epoch):
      self.set_theta((), -np.log(Z) )
      self.compute_P()
      self.compute_eta()
      sg=self.compute_squared_gradient()
      kl=self.compute_KL()
      #print(self.compute_ll(X))
      print(epoch ,":",  "KL divergence:",f'{kl:.8f}' ," time : %4.2f"% (time.time()-start),"Squared Gradient:",f'{sg:10f}')
      index = np.random.RandomState(seed=2019).permutation(range(len(self.B_)-1)) #exclude the bottom
     
      #compute etahat
      for iter in range(len(self.B_) -1):
        phi = self.B_[index[iter] +1 ]
        Ssub=self.Ssub_[phi]
        etahat_phi = self.etahat_[phi] 
        eta_phi = 0.0
        for x in Ssub:
            eta_phi += u[x]
        eta_phi /= Z

        sigma = eta_phi * Z
        tau = (1-eta_phi) * Z
        delta = np.log(tau/sigma * etahat_phi/(1-etahat_phi))


        step_base = step_init 
        new_eta_phi = eta_phi
        
        while np.abs(etahat_phi - new_eta_phi) > 0.0001:      
          step = step_base * ( self.etahat_[phi] - new_eta_phi )
          #print(etahat_phi - new_eta_phi, step)
          #
          step = delta
          
          u_new, Z_new = self.update_uZ(u, Z, step, phi)
          
          eta_phi_step  = 0.0
          for x in Ssub:
              eta_phi_step += u_new[x]
          eta_phi_step /= Z_new

          if np.abs(etahat_phi - new_eta_phi) >  np.abs(etahat_phi - eta_phi_step ): #if it gets better
            self.set_theta(phi, self.get_theta(phi) + step )
            new_eta_phi = eta_phi_step
            u, Z = u_new, Z_new            
          else: 
            step_base *= 0.3
     
      
  def update_uZ(self, u, Z, step, phi):
    """
    Updating u,Z
    Parameters
    -----
    u
    Z
    step
    phi
    
    """
    u_new = u.copy()
    Ssub=self.Ssub_[phi]
    for x in Ssub:
        u_new[x] *= np.exp(step)
    Z_new = Z
    for x in Ssub:
        Z_new += u[x] * (np.exp(step) -1)
    return u_new, Z_new
  
  def coordinate_descent2(self, X, max_epoch):  
    """
    Actual implementation gradient_descent
    Parameters 
    -----
    X : array-like, shape (n_samples,)
            Training vectors, where n_samples is the number of samples and 
            each element is a tuple of indices of 1s. 
    max_epoch
    """
    self.init_theta()
    start = time.time()
    self.compute_P()
    self.compute_eta()
    for epoch in range(max_epoch):
      
      print(epoch ,":",  "KL divergence:",f'{self.compute_KL():.8f}' ," time : %4.2f"% (time.time()-start),"Squared Gradient:",self.compute_squared_gradient())
      index = np.random.RandomState(seed=2019).permutation(range(len(self.B_)-1)) #exclude the bottom
      #index = range(len(self.B_)-1)
      for iter in range(len(self.B_) -1):
        
        phi = self.B_[index[iter] +1 ]
        #print(phi)
        s, t = self.compute_st(phi)
        d = np.log(t / s * self.etahat_[phi] / (1 - self.etahat_[phi]) )
        #print(d)
        new_theta_phi = self.get_theta(phi) + d
        self.set_theta(phi, new_theta_phi)
        self.theta_[self.invB_[()]] = -np.log(s * np.exp(d) + t)
        self.compute_P()
        self.compute_eta()
  
  def compute_st(self,phi):
    s = self.eta_[phi] * np.exp(-self.theta_[self.invB_[()]])
    t = (1 - self.eta_[phi]) * np.exp(-self.theta_[self.invB_[()]])
    
    return s,t
  


データ取得
===

In [3]:
!wget http://fimi.uantwerpen.be/data/chess.dat
!ls


--2019-09-22 05:17:05--  http://fimi.uantwerpen.be/data/chess.dat
Resolving fimi.uantwerpen.be (fimi.uantwerpen.be)... 143.129.69.1
Connecting to fimi.uantwerpen.be (fimi.uantwerpen.be)|143.129.69.1|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 342294 (334K)
Saving to: ‘chess.dat’

chess.dat           100%[===================>] 334.27K  --.-KB/s    in 0.07s   

2019-09-22 05:17:05 (4.97 MB/s) - ‘chess.dat’ saved [342294/342294]

chess.dat  sample_data


In [0]:
!wget http://fimi.uantwerpen.be/data/retail.dat
!ls

--2019-09-01 13:16:30--  http://fimi.uantwerpen.be/data/retail.dat
Resolving fimi.uantwerpen.be (fimi.uantwerpen.be)... 143.129.69.1
Connecting to fimi.uantwerpen.be (fimi.uantwerpen.be)|143.129.69.1|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4167490 (4.0M)
Saving to: ‘retail.dat’

retail.dat          100%[===================>]   3.97M  4.66MB/s    in 0.9s    

2019-09-01 13:16:32 (4.66 MB/s) - ‘retail.dat’ saved [4167490/4167490]

retail.dat  sample_data


In [21]:
X= []
m = 1000
n = -1
with open("chess.dat") as f:
  for line in f:
    xi =line.split()
    xi_int = [ int(xij)  for xij in xi ]
        
    if m > min(xi_int):
      m = min(xi_int)

    if n < max(xi_int):
          
      n = max(xi_int)
with open("chess.dat") as f:
  for line in f:
    xi = line.split()
    xi_int = [ int(xij) - m for xij in xi ]
    X.append(tuple(xi_int))
n = n-m+1
print(m)
print(X[0])

1
(0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 33, 35, 37, 39, 41, 43, 45, 47, 49, 51, 53, 55, 57, 59, 61, 63, 65, 67, 69, 71, 73)


その他

In [0]:
X =[ (0,1,2), (2,4), (4,)]

B = [(),(2,),(4,),(2,4)]
S = list(dict.fromkeys(B+X))
tbm = TBM(5, B, S, X)
tbm.fit(X,10,0.5)

gradient_descent
0 : KL divergence: 0.51082562  time : 0.00 Squared Gradient: 0.09333333333333328
1 : KL divergence: 0.46792210  time : 0.00 Squared Gradient: 0.06551792740676113
2 : KL divergence: 0.43788786  time : 0.00 Squared Gradient: 0.045547647532480105
3 : KL divergence: 0.41700812  time : 0.00 Squared Gradient: 0.03174533694167355
4 : KL divergence: 0.40241605  time : 0.01 Squared Gradient: 0.022446057233144846
5 : KL divergence: 0.39204449  time : 0.01 Squared Gradient: 0.016272982821671396
6 : KL divergence: 0.38446983  time : 0.01 Squared Gradient: 0.012199836444544544
7 : KL divergence: 0.37874062  time : 0.01 Squared Gradient: 0.00950869833520501
8 : KL divergence: 0.37423247  time : 0.01 Squared Gradient: 0.007716821501045386
9 : KL divergence: 0.37053960  time : 0.01 Squared Gradient: 0.006507378858732223


In [19]:
X =[ (0,1,2), (2,4), (4,) , (4,),  (4,), (4,) ]
B = [(),(2,),(4,),(2,4)]
S = list(dict.fromkeys(B+X))
tbm = TBM(5, B, S, X)
tbm.compute_Phat(X)
tbm.set_etahat(X)
tbm.coordinate_descent(X,10)

0 : KL divergence: 0.74187468  time : 0.00 Squared Gradient:   0.260000
1 : KL divergence: 0.19906580  time : 0.00 Squared Gradient:   0.005547
2 : KL divergence: 0.17489901  time : 0.01 Squared Gradient:   0.001375
3 : KL divergence: 0.16542358  time : 0.01 Squared Gradient:   0.001017
4 : KL divergence: 0.15822019  time : 0.01 Squared Gradient:   0.000766
5 : KL divergence: 0.15272948  time : 0.01 Squared Gradient:   0.000595
6 : KL divergence: 0.14841842  time : 0.01 Squared Gradient:   0.000474
7 : KL divergence: 0.14495751  time : 0.01 Squared Gradient:   0.000385
8 : KL divergence: 0.14212547  time : 0.02 Squared Gradient:   0.000318
9 : KL divergence: 0.13976994  time : 0.02 Squared Gradient:   0.000267


In [0]:
n = D.max().max() + 1 # including 0


import itertools

phi_all = list(itertools.combinations(range(n),1)) + list(itertools.combinations(range(n),2))

B = []
for phi in phi_all:
  etahat = 0.0
  for x in Y:
    if includes(phi, x):
      etahat += 1
#  etahat /= len(X)
  #print(etahat)
  if etahat > 0:
    B.append(phi)
print(len(B))
print(len(phi_all))
print(B)

NameError: ignored

Bをファイルから取得

In [2]:
from google.colab import files
files.upload()

ModuleNotFoundError: No module named 'google'

In [0]:
import pandas as pd
def get_B_from(filename):
  # file format:
  # each line is a frequent itemset with 1-start
  # ignore the first line that represents "bottom"
  # e.g.
  #   
  # 1 
  # 2
  # 3
  # 1 2
  # 1 3
  # 1 2 3
  Bfile = pd.read_csv(filename, header=None, sep=' ',names=[0,1])
  Bfile.head()
  B1=[(),]
  B2=[]
  for i in range(len(Bfile)):
    if pd.isnull(Bfile.at[i,1]): #要素が一つのものだけをB1に入れる
      B1.append((Bfile.loc[i,0]-1,)) #全ての要素-1

  for j in range(len(Bfile)):
    if pd.isnull(Bfile.at[j,1])==0: #要素が二つのものをB2に入れる
      B2.append((Bfile.loc[j,0]-1,int(Bfile.loc[j,1]-1))) #全ての要素-1
  B1.sort()
  B2.sort()
  B=B1+B2

  return B

In [0]:
B = get_B_from("chess.dat_itemsets")
S = list(dict.fromkeys(B+X))
tbm = TBM(n,B,S,X)

In [0]:
tbm.fit(X,2,0.01)

gradient_descent
0 : KL divergence: 0.11038542  time : 4.81 Squared Gradient: 2.029937764555959
1 : KL divergence: 0.09208482  time : 10.02 Squared Gradient: 1.323405437670419


In [9]:
tbm.compute_Phat(X)

tbm.coordinate_descent2(X,10)

AttributeError: ignored

In [24]:
tbm.compute_Phat(X)
tbm.set_etahat(X)
tbm.coordinate_descent(X,10)

0 : KL divergence: 0.11038542  time : 4.33 Squared Gradient:   2.029938
1 : KL divergence: 0.00644338  time : 12.24 Squared Gradient:   0.000673
2 : KL divergence: 0.00427300  time : 19.94 Squared Gradient:   0.000132
3 : KL divergence: 0.00370289  time : 27.40 Squared Gradient:   0.000065
4 : KL divergence: 0.00334168  time : 34.68 Squared Gradient:   0.000045
5 : KL divergence: 0.00308407  time : 41.78 Squared Gradient:   0.000034
6 : KL divergence: 0.00288866  time : 48.72 Squared Gradient:   0.000030
7 : KL divergence: 0.00272666  time : 55.66 Squared Gradient:   0.000025
8 : KL divergence: 0.00259071  time : 62.60 Squared Gradient:   0.000022
9 : KL divergence: 0.00247486  time : 69.42 Squared Gradient:   0.000020


In [0]:
A = [(1,2,3),(4,5,6),(7,8,9)]
print(A[0][1])
for i in range(len(A)):
  for j in range(len(A[i])):
    A[i][j] = A[i][j] -1
   

2


TypeError: ignored

In [4]:
!ls ../dataset

chess.dat	      kosarak.dat	     retail.dat
chess.dat_itemsets    kosarak.dat_itemsets   retail.dat_itemsets
connect.dat	      mushroom.dat
connect.dat_itemsets  mushroom.dat_itemsets
